## Classification House-types
#### Anforderungen:
<p> Entwickle und vergleiche drei sinnvolle Modelle zur Klassifikation von Immobilien Objekten hinsichtlich `type`. </p>
<p> Was sind sinnvolle Metriken zur Messung der Genauigkeit der Vorhersage im vorliegenden Fall? Was ist zu beachten um eine gute Abschätzung des Fehlers für neue Daten zu bekommen? </p>
<p> Rapportiere diese Metrik(en) mit einer Abschätzung des Fehlers für alle drei Modelle </p>

In [124]:
# import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn import model_selection, svm, ensemble, tree
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import svm
import random


In [125]:
# import dataset
df = pd.read_csv('../data/model/immoscout_robust.csv')
df.tail(5)

,ForestDensityM,Latitude,Longitude,NoisePollutionRailwayM,NoisePollutionRoadM,PopulationDensityM,RiversAndLakesM,RiversAndLakesS,distanceToTrainStation,gde_area_agriculture_percentage,...,type_penthouse,type_rustico,type_secondary-suite,type_semi-detached-house,type_single-room,type_stepped-apartment,type_stepped-house,type_studio,type_terrace-house,type_villa
13308,-0.214762,0.614418,0.482482,0.0,-0.066882,0.825061,0.000000,0.0,-0.007174,0.214048,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13309,0.031322,0.659631,0.467615,0.0,0.684524,0.537781,108.488669,0.0,0.527702,0.111050,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
13310,-0.214762,0.812664,0.476216,0.0,-0.005971,0.720047,0.000000,0.0,-0.019672,0.789850,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
13311,0.291106,0.703465,0.506514,0.0,0.060185,0.243838,0.000000,0.0,0.896919,0.279233,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13312,0.521797,0.755503,0.512116,0.0,-1.150419,0.255263,0.000000,0.0,-0.292978,-0.287039,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
# add a new column with different types
types = [col for col in df.columns if 'type' in col]
df['type'] = df[types].idxmax(axis=1)
df['type'] = df['type'].str.replace('type_', '')

In [127]:
df.columns

Index(['ForestDensityM', 'Latitude', 'Longitude', 'NoisePollutionRailwayM',
       'NoisePollutionRoadM', 'PopulationDensityM', 'RiversAndLakesM',
       'RiversAndLakesS', 'distanceToTrainStation',
       'gde_area_agriculture_percentage', 'gde_area_forest_percentage',
       'gde_area_nonproductive_percentage', 'gde_average_house_hold',
       'gde_empty_apartments', 'gde_foreigners_percentage',
       'gde_new_homes_per_1000', 'gde_politics_bdp', 'gde_politics_cvp',
       'gde_politics_evp', 'gde_politics_fdp', 'gde_politics_glp',
       'gde_politics_gps', 'gde_politics_pda', 'gde_politics_rights',
       'gde_politics_sp', 'gde_politics_svp', 'gde_pop_per_km2',
       'gde_population', 'gde_social_help_quota', 'gde_tax',
       'gde_workers_sector1', 'rooms', 'Floor space_m2', 'Plot area_m2',
       'living_space_m2', 'price', 'type_attic-flat', 'type_attic-room',
       'type_castle', 'type_chalet', 'type_detached-house',
       'type_detached-secondary-suite', 'type_duplex-mais

In [128]:
#drop type-columns for the features
types.append('type')
types

['type_attic-flat',
 'type_attic-room',
 'type_castle',
 'type_chalet',
 'type_detached-house',
 'type_detached-secondary-suite',
 'type_duplex-maisonette',
 'type_farmhouse',
 'type_flat',
 'type_furnished-residential-property',
 'type_loft',
 'type_penthouse',
 'type_rustico',
 'type_secondary-suite',
 'type_semi-detached-house',
 'type_single-room',
 'type_stepped-apartment',
 'type_stepped-house',
 'type_studio',
 'type_terrace-house',
 'type_villa',
 'type']

In [129]:
X = df.drop(types, axis=1)
y = df['type']
df = df.drop(types, axis=1)

In [139]:
df.columns

Index(['ForestDensityM', 'Latitude', 'Longitude', 'NoisePollutionRailwayM',
       'NoisePollutionRoadM', 'PopulationDensityM', 'RiversAndLakesM',
       'RiversAndLakesS', 'distanceToTrainStation',
       'gde_area_agriculture_percentage', 'gde_area_forest_percentage',
       'gde_area_nonproductive_percentage', 'gde_average_house_hold',
       'gde_empty_apartments', 'gde_foreigners_percentage',
       'gde_new_homes_per_1000', 'gde_politics_bdp', 'gde_politics_cvp',
       'gde_politics_evp', 'gde_politics_fdp', 'gde_politics_glp',
       'gde_politics_gps', 'gde_politics_pda', 'gde_politics_rights',
       'gde_politics_sp', 'gde_politics_svp', 'gde_pop_per_km2',
       'gde_population', 'gde_social_help_quota', 'gde_tax',
       'gde_workers_sector1', 'rooms', 'Floor space_m2', 'Plot area_m2',
       'living_space_m2', 'price'],
      dtype='object')

In [130]:
def train_test_split(X, y):
    return model_selection.train_test_split(X, y, train_size=0.6, random_state=42)

In [131]:
#select random columns for the features
def get_random_column_names(columns: list, percentage: float, add_type_columns: bool) -> list:
    type_columns = [c for c in columns if c.startswith("type_") and add_type_columns]
    regular_columns = [c for c in columns if (not c.startswith("type_")) and random.random() < percentage]
    return [*type_columns, *regular_columns]

In [132]:
# K-nearest-neighbour
def knn(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series) -> dict:
    model = KNeighborsClassifier(n_neighbors= 5)
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)

    return {
        "type": "K-Nearest-Neighbour",
        "columns": list(X_train.columns),
        "num_columns": len(X_train.columns),
        "score": model.score(X_test, y_test),
        'accuracy': accuracy_score(y_test, y_predict),
        "model": model
    }

In [133]:
# Support Vector Machine other kernels should be tried
def svm(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series) -> dict:
    model = SVC(kernel= 'rbf')
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)

    return {
        "type": "SVM",
        "columns": list(X_train.columns),
        "num_columns": len(X_train.columns),
        "score": model.score(X_test, y_test),
        'accuracy': accuracy_score(y_test, y_predict),
        "model": model
    }

In [134]:
# Random Forest Classifier max depth and other coefficients should be tried
def random_forest(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series) -> dict:
    model = ensemble.RandomForestClassifier(n_estimators=50, criterion = 'gini', random_state=42)
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)

    return {
        "type": "RandomForest",
        "columns": list(X_train.columns),
        "num_columns": len(X_train.columns),
        "score": model.score(X_test, y_test),
        'accuracy': accuracy_score(y_test, y_predict),
        "model": model
    }

In [135]:

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [136]:
# Constants
NUM_ITERATIONS = 2
MIN_PERCENTAGE_COLUMNS = 0.2
MAX_PERCENTAGE_COLUMNS = 1
TRAINING_FUNCTIONS = [knn, svm, random_forest]

print("Total number of iterations:", NUM_ITERATIONS * len(TRAINING_FUNCTIONS) * 2)

Total number of iterations: 12


In [137]:
X.columns

Index(['ForestDensityM', 'Latitude', 'Longitude', 'NoisePollutionRailwayM',
       'NoisePollutionRoadM', 'PopulationDensityM', 'RiversAndLakesM',
       'RiversAndLakesS', 'distanceToTrainStation',
       'gde_area_agriculture_percentage', 'gde_area_forest_percentage',
       'gde_area_nonproductive_percentage', 'gde_average_house_hold',
       'gde_empty_apartments', 'gde_foreigners_percentage',
       'gde_new_homes_per_1000', 'gde_politics_bdp', 'gde_politics_cvp',
       'gde_politics_evp', 'gde_politics_fdp', 'gde_politics_glp',
       'gde_politics_gps', 'gde_politics_pda', 'gde_politics_rights',
       'gde_politics_sp', 'gde_politics_svp', 'gde_pop_per_km2',
       'gde_population', 'gde_social_help_quota', 'gde_tax',
       'gde_workers_sector1', 'rooms', 'Floor space_m2', 'Plot area_m2',
       'living_space_m2', 'price'],
      dtype='object')

In [138]:
results = []
for _ in range(NUM_ITERATIONS):
    for add_type_columns in [True, False]:
        column_percentage = random.random() * (MAX_PERCENTAGE_COLUMNS - MIN_PERCENTAGE_COLUMNS) + MIN_PERCENTAGE_COLUMNS
        column_names = get_random_column_names(X_train.columns, column_percentage, add_type_columns)
        temp_X_train, temp_X_test = X_train[column_names], X_test[column_names]

        for func in TRAINING_FUNCTIONS:
            results.append({**func(temp_X_train, temp_X_test, y_train, y_test), "add_type_columns": add_type_columns})

pd.DataFrame(results).sort_values("score", ascending=False)

,type,columns,num_columns,score,accuracy,model,add_type_columns
5,RandomForest,"[Latitude, Longitude, NoisePollutionRoadM, Pop...",30,0.647953,0.647953,"(DecisionTreeClassifier(max_features='sqrt', r...",False
8,RandomForest,"[ForestDensityM, Longitude, NoisePollutionRoad...",26,0.606083,0.606083,"(DecisionTreeClassifier(max_features='sqrt', r...",True
2,RandomForest,"[Longitude, NoisePollutionRoadM, PopulationDen...",23,0.519151,0.519151,"(DecisionTreeClassifier(max_features='sqrt', r...",True
0,K-Nearest-Neighbour,"[Longitude, NoisePollutionRoadM, PopulationDen...",23,0.486481,0.486481,KNeighborsClassifier(),True
4,SVM,"[Latitude, Longitude, NoisePollutionRoadM, Pop...",30,0.485355,0.485355,SVC(),False
7,SVM,"[ForestDensityM, Longitude, NoisePollutionRoad...",26,0.485355,0.485355,SVC(),True
11,RandomForest,"[gde_new_homes_per_1000, gde_politics_fdp, gde...",5,0.481224,0.481224,"(DecisionTreeClassifier(max_features='sqrt', r...",False
3,K-Nearest-Neighbour,"[Latitude, Longitude, NoisePollutionRoadM, Pop...",30,0.478783,0.478783,KNeighborsClassifier(),False
6,K-Nearest-Neighbour,"[ForestDensityM, Longitude, NoisePollutionRoad...",26,0.470710,0.470710,KNeighborsClassifier(),True
10,SVM,"[gde_new_homes_per_1000, gde_politics_fdp, gde...",5,0.463763,0.463763,SVC(),False


In [143]:
results[0]

{'type': 'K-Nearest-Neighbour',
 'columns': ['Longitude',
  'NoisePollutionRoadM',
  'PopulationDensityM',
  'RiversAndLakesM',
  'RiversAndLakesS',
  'distanceToTrainStation',
  'gde_area_agriculture_percentage',
  'gde_area_forest_percentage',
  'gde_area_nonproductive_percentage',
  'gde_average_house_hold',
  'gde_foreigners_percentage',
  'gde_politics_bdp',
  'gde_politics_cvp',
  'gde_politics_evp',
  'gde_politics_glp',
  'gde_politics_rights',
  'gde_politics_sp',
  'gde_politics_svp',
  'gde_population',
  'gde_social_help_quota',
  'gde_tax',
  'gde_workers_sector1',
  'rooms'],
 'num_columns': 23,
 'score': 0.4864814119414195,
 'accuracy': 0.4864814119414195,
 'model': KNeighborsClassifier(),
 'add_type_columns': True}

## Offene Punkte:
Random Forest domminiert

try different kernel, forste depth etc...

kontrolle ob type berücksichtigt werden